Load data

In [4]:
import json


file_path = '12-11-game-states.json'

game_states = []
with open(file_path, 'r') as file:
    for line in file:
        game_states.append(json.loads(line.strip()))

print(f"Loaded {len(game_states)} game states.")

Loaded 12659 game states.


In [5]:
print(game_states[0])

{'turn': '1', 'player': {'active_pokemon': {'mon': 'Mesprit', 'hp_percent': 0.68364, 'status': [1, 0, 0, 0, 0, 0], 'terratype': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'boosts': {'atk': 0, 'def': 0, 'spa': 0, 'spd': 0, 'spe': 0, 'evasion': 0, 'accuracy': 0}, 'player_side_effects': {'Stealth': 0, 'Spikes': 0, 'Toxic': 0, 'Sticky': 0, 'Reflect': 0, 'reflect_turns': 0, 'tailwind': 0, 'tailwind_turns': 0}, 'last_move': None, 'next_move': 'Thunder Wave', 'num_mons_left': 6, 'seen_pokemon': {'poke1': {'mon': 'Ludicolo', 'hp_percent': 1.0, 'status': [1, 0, 0, 0, 0, 0], 'terratype': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'poke2': {'mon': 'Dondozo', 'hp_percent': 1.0, 'status': [1, 0, 0, 0, 0, 0], 'terratype': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'poke3': {'mon': 'Mesprit', 'hp_percent': 1.0, 'status': [1, 0, 0, 0, 0, 0], 'terratype': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'poke4': {'mon': 'Pikachu-Unova', 'hp_percent': 1.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Define embedding parameters
name_vocab_size = 509  # Update based on unique Pokémon names
embedding_dim = 32      # Tune based on dataset size

# Input layers for player and opponent Pokémon
def create_pokemon_input(prefix):
    name_input = Input(shape=(1,), name=f"{prefix}_name")  # Pokémon name
    hp_input = Input(shape=(1,), name=f"{prefix}_hp")  # HP percentage
    status_input = Input(shape=(6,), name=f"{prefix}_status")  # Status one-hot
    terra_input = Input(shape=(18,), name=f"{prefix}_terra")  # Terra type

    # Embedding for Pokémon name
    name_embedding = layers.Embedding(name_vocab_size, embedding_dim, name=f"{prefix}_name_embedding")(name_input)
    name_embedding = layers.Flatten()(name_embedding)

    # Concatenate all inputs
    combined = layers.Concatenate()([name_embedding, hp_input, status_input, terra_input])
    return name_input, hp_input, status_input, terra_input, combined

# Create inputs for active Pokémon and seen Pokémon (both player and opponent)
player_inputs = []
opponent_inputs = []
player_combined = []
opponent_combined = []

for i in range(6):  # Six Pokémon slots
    pname, php, pstatus, pterra, pcomb = create_pokemon_input(f"player_poke{i+1}")
    oname, ohp, ostatus, oterra, ocomb = create_pokemon_input(f"opponent_poke{i+1}")

    player_inputs.extend([pname, php, pstatus, pterra])
    opponent_inputs.extend([oname, ohp, ostatus, oterra])
    player_combined.append(pcomb)
    opponent_combined.append(ocomb)

# Combine Pokémon data
player_features = layers.Concatenate()(player_combined)
opponent_features = layers.Concatenate()(opponent_combined)

# Add side effects, boosts, and battle state features
side_effects = Input(shape=(10,), name="side_effects")
boosts = Input(shape=(7,), name="boosts")
battle_state = Input(shape=(9,), name="battle_state")

# Combine all features
combined_features = layers.Concatenate()([player_features, opponent_features, side_effects, boosts, battle_state])

# Dense layers
x = layers.Dense(128, activation="relu")(combined_features)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)

# Output layer
output = layers.Dense(1, activation="sigmoid", name="win_rate")(x)

# Define the model
model = Model(inputs=player_inputs + opponent_inputs + [side_effects, boosts, battle_state], outputs=output)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()
